# Imports 

In [2]:
import pandas as pd
import numpy as np
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Read in data and make sure relevant columns are string/character 

- San Diego data: `naics_code` and `account_key`
- NAICS details data: `naics` (North American Industry Classification Systems)

Run code below; if pulling from github, pathname should be fine; if working elsewhere may need to edit path name at read in 

In [5]:
# uncomment if u need to find your path: os.getcwd()
SD = sd_df = pd.read_csv("../public_data/sd_df.csv")
NA = naics_df = pd.read_csv("../public_data/naics_df.csv")

In [7]:
cols_sd_use = ["naics_code", "account_key"]
cols_naics_use = ["naics"]

sd_df[cols_sd_use] = sd_df[cols_sd_use].astype(str)
naics_df[cols_naics_use] = naics_df[cols_naics_use].astype(str)

sd_df.dtypes
naics_df.dtypes
sd_df.head()
naics_df.head()
sd_df.shape,naics_df.shape

account_key          object
dba_name             object
council_district     object
naics_code           object
naics_description    object
naics_nchar           int64
dtype: object

naics                object
naics_description    object
dtype: object

,account_key,dba_name,council_district,naics_code,naics_description,naics_nchar
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4


,naics,naics_description
0,111140,Wheat Farming
1,111160,Rice Farming
2,111150,Corn Farming
3,111110,Soybean Farming
4,111120,Oilseed (except Soybean) Farming


((353, 6), (2270, 2))

## "Inner join"- retain only San Diego businesses with details on their NAICS code

- Use the `naics_code` column in the San Diego business data as the join key
- Use the `naics` column in the NAICS code details data as the join key

- Do an inner join of the San Diego data onto the NAICS code details using these join keys
- After the inner join, print some examples of San Diego businesses lost in the merge
- Use value_counts() on the `naics_nchar` column in the San Diego data to see why they might have gotten lost


In [7]:
# your code here
naics_df = pd.merge(naics_df, sd_df, left_on= "naics", right_on= "naics_code", how="inner")
naics_df.shape

(52, 8)

In [8]:
sd_df

,account_key,dba_name,council_district,naics_code,naics_description,naics_nchar
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4
...,...,...,...,...,...,...
348,2007015303,WFP-GATEWAY LP,cd_3,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3
349,2007015336,WFP-MIRAMAR LP,cd_3,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3
350,2010010820,ROSSLYN LOFTS HOUSING PARTNERS LP THE,cd_6,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3
351,2014004525,INSTANT SMOG CHECK,cd_6,811198,ALL OTHER AUTOMOTIVE R&M,6


In [9]:
missing_accounts_df = sd_df[~sd_df["account_key"].isin(naics_df["account_key"])]
missing_accounts_df["naics_nchar"].value_counts()

naics_nchar
5    150
4     89
3     42
2     23
6     14
Name: count, dtype: int64

## "Left join"- retain all sd businesses even if naics code isn't in `naics_df`

- Using the same join keys as above, and treating the San Diego businesses as the left hand side data, left join the naics code details onto the San Diego businesses
- Use the `indicator` argument within merge to create an indicator, `naics_merge_status`, to help with later merge diagnostics and examine sample of ones that didn't merge
- Use the `suffixes` argument within merge to add `_sd` as the left suffix, `_census` as the right suffix
- Use `naics_merge_status` in the merged result to look at a sample of San Diego businesses that weren't matched with `naics_df`

In [9]:
# your code here
left_join = pd.merge(sd_df, naics_df, how="left", left_on= "naics_code" , 
                        right_on= "naics",
                        indicator="naics_merge_status", 
                        suffixes=("_sd","_census"))

In [11]:
left_join

,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,naics,naics_description_census,naics_merge_status
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,Offices of Certified Public Accountants,both
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,Offices of Certified Public Accountants,both
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...
365,2010010820,ROSSLYN LOFTS HOUSING PARTNERS LP THE,cd_6,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3,NaN,NaN,left_only
366,2014004525,INSTANT SMOG CHECK,cd_6,811198,ALL OTHER AUTOMOTIVE R&M,6,811198,All Other Automotive Repair and Maintenance,both
367,2014004525,INSTANT SMOG CHECK,cd_6,811198,ALL OTHER AUTOMOTIVE R&M,6,811198,All Other Automotive Repair and Maintenance,both
368,2014004525,INSTANT SMOG CHECK,cd_6,811198,ALL OTHER AUTOMOTIVE R&M,6,811198,All Other Automotive Repair and Maintenance,both


In [13]:
#left_join[left_join["naics_merge_status"] == "left_only"].sample(10)

## Use group by and agg to see if there are differences in merge rates by area

- Using the left-joined dataframe created in previous step, create a boolean indicator, `is_lost`, that equals `True` if the merge indicator is equal to "left_only" (and is otherwise false)
- Find the proportion of businesses by council district that were lost in the left join (SD businesses that failed to match to `naics_df`). To do this, group by `council_district` and use the shortcut of taking the mean of the `is_lost` indicator to find the proportions lost by `council_district`. 

In [16]:
# your code here
left_join["is_lost"] = (left_join["naics_merge_status"] == "left_only")

In [18]:
left_join

,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,naics,naics_description_census,naics_merge_status,is_lost
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,Offices of Certified Public Accountants,both,False
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only,True
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,Offices of Certified Public Accountants,both,False
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only,True
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4,NaN,NaN,left_only,True
...,...,...,...,...,...,...,...,...,...,...
365,2010010820,ROSSLYN LOFTS HOUSING PARTNERS LP THE,cd_6,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3,NaN,NaN,left_only,True
366,2014004525,INSTANT SMOG CHECK,cd_6,811198,ALL OTHER AUTOMOTIVE R&M,6,811198,All Other Automotive Repair and Maintenance,both,False
367,2014004525,INSTANT SMOG CHECK,cd_6,811198,ALL OTHER AUTOMOTIVE R&M,6,811198,All Other Automotive Repair and Maintenance,both,False
368,2014004525,INSTANT SMOG CHECK,cd_6,811198,ALL OTHER AUTOMOTIVE R&M,6,811198,All Other Automotive Repair and Maintenance,both,False


In [22]:
left_join.groupby("council_district")["is_lost"].agg("mean")

council_district
cd_1    0.880597
cd_2    0.804348
cd_3    0.812500
cd_4    1.000000
cd_5    0.916667
cd_6    0.859649
cd_7    0.918919
cd_8    0.933333
cd_9    1.000000
Name: is_lost, dtype: float64

## Optional challenge exercise: add lagging 0's and see if merge rate improves

You noticed earlier that a big reason for non-matches is that the San Diego tax certificate NAICS codes were often less than six digits long, while the Census ones were always 6 digits.

You wonder if this is an issue where 0's in some of the SD's data naics codes got cut off (eg 540000 became 54), and if so whether adding these lagging zeros would improve the merge rate in a left join.

- Pad the `naics_code` column in `sd_df` with 0's to get that column up to 6-digits, using one of two approaches: 
  1. `str.pad` in pandas (https://pandas.pydata.org/docs/reference/api/pandas.Series.str.pad.html)
  2. for more of a challenge, write your own function! It should check the # of digits in the naics code string and pad it with 0's at the end up to 6 digits. To execute your function, use row-wise apply: `df.apply(lambda row: funcname(row.column), axis=1)`.
- Perform the same left join as above and see how the match rate changes
- Create an indicator variable--`is_new_match`---for new matches under the padded NAICS code; compare the `naics_description` column from San Diego versus Census in the new dataset for a sample of these new matches and comment on whether the padding seems to be correct

In [26]:
# your code here
sd_df["naics_code"].str.pad(6,side="right",fillchar="0")
new_join = pd.merge(sd_df, naics_df, how="left", left_on= "naics_code" , 
                        right_on= "naics",
                        indicator="naics_merge_status", 
                        suffixes=("_sd","_census"))


0      541211
1      541100
2      541211
3      541100
4      541200
        ...  
348    541000
349    541000
350    541000
351    811198
352    531000
Name: naics_code, Length: 353, dtype: object

In [42]:
new_join["is_new_match"] = (new_join["naics_merge_status"] == "both")

In [44]:
new_join.groupby("council_district")["is_lost"].agg("mean")

KeyError: 'Column not found: is_lost'